In [26]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.preprocessing import StandardScaler
import re

# 데이터 로드
df = pd.read_csv('coupang_review_merged.csv', encoding='utf-8-sig')

# 불필요한 문자 제거 함수
def clean_text(text):
    # 문자열 타입으로 변환 시 인코딩 처리
    text = str(text).encode('utf-8', errors='ignore').decode('utf-8')
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [27]:
okt = Okt()

def normalize_text(text):
    # 한글 형태소 분석
    morphs = okt.morphs(text, stem=True)
    
    # 한글 불용어 정의
    korean_stopwords = [
        '이', '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나',
        '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하', '오',
        '말', '일', '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받', '못하',
        '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', '중',
        '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집',
        '살', '모르', '적', '월', '데', '자신', '안', '어떤', '내', '경우', '명', '생각',
        '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻', '여자',
        '개', '전', '들', '사실', '이렇', '점', '싶', '말', '정도', '좀', '원', '잘', '걍',
        'ㅋ', 'ㅎ', 'ㅠ', 'ㅜ', '음', '아', '어', '아니', '구매', '상품', '제품'
    ]
    
    # 불용어 제거 및 정규화
    morphs = [word for word in morphs if word not in korean_stopwords and len(word) > 1]
    return ' '.join(morphs)

In [28]:
def extract_features(df):
    # 텍스트 기본 특징
    df['text_length'] = df['내용'].str.len()
    df['word_count'] = df['내용'].str.split().str.len()
    
    # 특수문자 개수
    df['special_char_count'] = df['내용'].str.count(r'[!?~♡❤️★⭐️✨]')
    
    # 이모티콘 개수
    df['emoticon_count'] = df['내용'].str.count(r'[ㅋㅎㅠㅜㅡㅍㅎㄷ]+')
    
    # 대문자 비율
    df['uppercase_ratio'] = df['내용'].apply(lambda x: sum(1 for c in str(x) if c.isupper()) / len(str(x)) if len(str(x)) > 0 else 0)
    
    # 문장 당 평균 단어 수
    df['avg_words_per_sentence'] = df['내용'].apply(lambda x: len(str(x).split()) / (str(x).count('.') + str(x).count('!') + str(x).count('?') + 1))
    
    return df

In [29]:
from konlpy.tag import Okt
from collections import Counter

def extract_sentiment_features(text):
    # 긍정/부정 키워드 정의
    positive_words = ['좋', '훌륭', '최고', '만족', '추천', '굿', '역대급', '완벽']
    negative_words = ['별로', '실망', '최악', '구려', '후회', '비추', '아쉽', '불만']
    
    # 형태소 분석
    morphs = okt.morphs(text)
    
    # 감성 점수 계산
    positive_score = sum([1 for word in morphs if word in positive_words])
    negative_score = sum([1 for word in morphs if word in negative_words])
    
    return positive_score, negative_score

In [30]:
def preprocess_data(df):
    print("데이터 전처리 시작...")
    
    # 1. 중복 제거
    df = df.drop_duplicates(subset=['내용'])
    print(f"중복 제거 후 데이터 크기: {df.shape}")
    
    # 2. 결측치 처리
    df = df.dropna()
    print(f"결측치 제거 후 데이터 크기: {df.shape}")
    
    # 3. 텍스트 정제
    df['cleaned_text'] = df['내용'].apply(clean_text)
    
    # 4. 텍스트 정규화
    df['normalized_text'] = df['cleaned_text'].apply(normalize_text)
    
    # 5. 특징 추출
    df = extract_features(df)
    
    # 6. 감성 분석 특징 추출
    sentiment_features = df['cleaned_text'].apply(extract_sentiment_features)
    df['positive_score'] = [score[0] for score in sentiment_features]
    df['negative_score'] = [score[1] for score in sentiment_features]
    
    # 7. 수치형 특징 정규화
    numeric_features = ['text_length', 'word_count', 'special_char_count', 
                       'emoticon_count', 'uppercase_ratio', 'avg_words_per_sentence',
                       'positive_score', 'negative_score']
    
    scaler = StandardScaler()
    df[numeric_features] = scaler.fit_transform(df[numeric_features])
    
    return df

In [31]:
if __name__ == "__main__":
    # 데이터 로드
    df = pd.read_csv('coupang_review_merged.csv', encoding='utf-8')
    
    # 데이터 전처리
    processed_df = preprocess_data(df)
    
    # TF-IDF 벡터화
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer(max_features=5000)
    X_tfidf = vectorizer.fit_transform(processed_df['normalized_text'])
    
    # 특징 결합
    numeric_features = ['text_length', 'word_count', 'special_char_count', 
                       'emoticon_count', 'uppercase_ratio', 'avg_words_per_sentence',
                       'positive_score', 'negative_score']
    
    X = pd.concat([
        pd.DataFrame(X_tfidf.toarray()),
        processed_df[numeric_features]
    ], axis=1)
    
    # 전처리된 데이터 저장
    processed_df.to_csv('processed_reviews.csv', encoding='utf-8-sig', index=False)
    print("저장완료")

데이터 전처리 시작...
중복 제거 후 데이터 크기: (10758, 6)
결측치 제거 후 데이터 크기: (10758, 6)
저장완료
